In [4]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg

# Given functions
def q(x):
    return 2 * np.cos(x)

def k(x):
    return 2 * np.sin(2 * x)

# Domain and discretization
n = 41
A, B = 0, 1  # Domain
x_j = np.linspace(A, B, n)
h = x_j[1] - x_j[0]  # Assuming uniform spacing

# Real function (for calculating mu1 and mu2)
real = lambda x: np.sin(np.pi * x)

# Boundary condition parameters
alpha1 = 1
alpha2 = 1

# Calculate mu1 and mu2 using the boundary conditions and the real solution
def derivative_of_real(x):
    # Derivative of the real solution
    return np.pi * np.cos(np.pi * x)

mu1 = -k(A) * derivative_of_real(A) + alpha1 * real(A)
mu2 = k(B) * derivative_of_real(B) + alpha2 * real(B)

mu1, mu2

(0.0, -5.713284232087328)

In [5]:
def find_phi(x, j, x_j, h):
    """ Linear shape function for the j-th node at position x. """
    if j == 0:
        if x < x_j[0] or x > x_j[1]:
            return 0
        else:
            return (x_j[j + 1] - x) / h
    elif j == n - 1:
        if x > x_j[-1] or x < x_j[-2]:
            return 0
        else:
            return (x - x_j[j - 1]) / h
    elif x < x_j[j] - h or x > x_j[j] + h:
        return 0
    elif x <= x_j[j]:
        return (x - x_j[j - 1]) / h
    else:
        return (x_j[j + 1] - x) / h

def assemble_stiffness_matrix(n, x_j, h):
    """ Assemble the stiffness matrix. """
    K = sp.lil_matrix((n, n))

    for i in range(n):
        for j in range(max(0, i - 1), min(n, i + 2)):  # Only consider neighboring nodes
            # Integrate using numerical integration (midpoint rule for simplicity)
            if i == j:
                # Main diagonal
                integral = h * (2 * k(x_j[i]) / h**2 + q(x_j[i]))
            else:
                # Off diagonal
                integral = -k(x_j[i]) / h**2

            K[i, j] += integral

    return K.tocsr()

def assemble_load_vector(n, x_j, h, mu1, mu2):
    """ Assemble the load vector. """
    F = np.zeros(n)

    # Using midpoint rule for numerical integration
    for i in range(n):
        F[i] = h * 2 * np.cos(x_j[i])  # f(x) = 2cos(x)

    # Adjusting for boundary conditions
    F[0] -= mu1
    F[-1] -= mu2

    return F

# Assemble stiffness matrix and load vector
K = assemble_stiffness_matrix(n, x_j, h)
F = assemble_load_vector(n, x_j, h, mu1, mu2)

# Solve the linear system KU = F
U = scipy.sparse.linalg.spsolve(K, F)

U 

array([ 1.        , -1.62085527, -1.08186187,  1.56643675,  1.16024276,
       -1.50841207, -1.23582055,  1.44650416,  1.30816579, -1.38108375,
       -1.37730336,  1.31214351,  1.44291952, -1.23998583, -1.50497361,
        1.16468007,  1.56321113, -1.08650822, -1.61757607,  1.0055824 ,
        1.66785529, -0.92217875, -1.71399419,  0.83643848,  1.75581376,
       -0.74863733, -1.79326898,  0.65893782,  1.82621161, -0.56761722,
       -1.85461094,  0.47485396,  1.87834777, -0.380927  , -1.89740881,
        0.28602643,  1.91170286, -0.1904322 , -1.92123612,  0.09434234,
        1.92594469])